# Chapter 12: Inheritance: For Good or For Worse

This chapter is about inheritance and subclassing, discussing good and bad practices when building class hierarchies. Chapter 12 emphasises two particulars very specific to Python:

* The pitfalls of subclassing from built-in types.

* Multiple inheritance and the method resolution order.

Multiple inheritance is illustrated using two important Python projects: the TKinter GUI toolkit and the Django Web framework. In this summary we do not repeat these examples, but instead give a high-level overview. Refer to the book for details.

## Subclassing Built-In Types is Tricky

Since Python 2.2 it has been possible to subclass built-in types such as `list` or `dict`. The is **one major caveat**: the code of the built-ins (written in C) does not all special methods overriden by user-defined classes.

In [1]:
# Example 12-1. __setitem__ override is ignored by __init__ and __update__

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': 1}

In [2]:
dd["two"] = 2
dd

{'one': 1, 'two': [2, 2]}

In [3]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

#### Things to note about the above implementation:

* `[]` calls our `__setitem__` and works as expected.

* The `update` method from `dict` does not use our version of `__setitem__`.

* This built-in behaviour violates a basic rule of OOP: the search for methods should always start from the class of the target instance (`self`), even when the call happens inside a method implemented in a superclass.

**Takeaway:** Subclassing built-ins is error-prone. Instead, derive classes from `collections` module using `UserDict`, `UserList`, and `UserString` which are designed to be easily extended.

In [4]:
# Example 12-3. DoppelDict now works as expected because it extends UserDict and not dict

import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': [1, 1]}

In [5]:
dd["two"] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [6]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

## Multiple Inheritance and Method Resolution Order

Any language implementing multiple inheritance needs to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name. This is called the **"diamond problem"**.

![](../figs/uml-diamondproblem.png)

In [7]:
# Example 12-4. classes A, B, C, and D for the graph in the above figure

class A:
    def ping(self):
        print("ping", self)
        
class B(A):
    def pong(self):
        print("pong", self)
        
class C(A):
    def pong(self):
        print("PONG", self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print("post-ping", self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        

In [8]:
d = D()
d.pong()

pong <__main__.D object at 0x104327340>


In [9]:
C.pong(d)

PONG <__main__.D object at 0x104327340>


#### Things to note about the above implementation

* Both `B` and `C` implement a `pong` method.

* You can always call a method on a superclass directly, passing the instance as an explicit argument (`C.pong(d)`).

* The ambiguity of the call `d.pong()`(which `pong()` method does it call, `B` or `C`?) is resolved because Python follows a specific order when traversing the inheritance graph.

* The order is called **MRO (method resolution order)**. This is stored in the `__mro__` special method, which holds a tuple of references to the superclasses in MRO order.

In [10]:
repr(D.__mro__)

"(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)"

The recommended way to delegate method calls to superclasses is the `super()` built-in function.

Sometimes it is convenient to bypass the MRO and invoke a method on a superclass directly. In this case, `self` must be passed explicitly since we are accessing an *unbound method*.

In [11]:
d = D()
d.ping()

ping <__main__.D object at 0x104327430>
post-ping <__main__.D object at 0x104327430>


`ping` of `D` makes two calls:

1. `super().ping()`, which delegates the call to class `A`.

2. `print("post-ping", self)` which outputs this line.

In [12]:
d = D()
d.pingpong()

ping <__main__.D object at 0x104344670>
post-ping <__main__.D object at 0x104344670>
ping <__main__.D object at 0x104344670>
pong <__main__.D object at 0x104344670>
pong <__main__.D object at 0x104344670>
PONG <__main__.D object at 0x104344670>


`pingpong` of `D` makes five calls:

1. `self.ping()` which runs `ping` method of `D`.

2. `super().ping()` which bypasses `ping` in `D` and delegates to `A`.

3. `self.pong()` which finds the `B` implementation of `pong`, according to `__mro__`.

4. `super().pong()` which finds the `B.pong` implementation, also following `__mro__`.

5. `C.pong(self)`, which finds `C.pong`, ignoring `__mro__`.

The MRO not only takes into account the inheritance graph, but also the order in which superclasses are listed in a subclass declaration.

If the `D` class where declared `D(C, B)`, the `__mro__` of class `D` would be different: `C` would be searched before `B`.

## Multiple Inheritance in the Real World

It's easy to create brittle and incomprehensible design using multiple inheritance. Here are som tips to avoid spaghetti class graphs.

### 1. Distinguish Interface Inheritance from Implementation Inheritance

The main reasons for subclassing is:

1. Inheritance of interface creates a subtype, implying "is-a" relationship.

2. Inheritance of implementation avoids duplication by reuse.

Whenever possible, make the intent clear. Inheritance for code reuse is an implementation detail and can often be replaced by composition and delegation. On the other hand, interface inheritance is the backbone of a framework.

### 2. Make Interfaces Explicit with ABCs

If a class is designed to define an interface, it should be an explicit ABC. In other words, subclass `abc.ABC` or another ABC.

### 3. Use Mixins for Code Reuse

If a class is designed to provide method implementations for reuse by multiple  unrelated subclasses, without implying an "is-a" relationship, it should be an explicit **mixin class**. 

Mixins bundle methods for reuse. They should never be instantiated, and concrete classes should not inherit only from a mixin. Each mixin should provide a single specific behaviour, implementing few and very closely related methods.

### 4. Make Mixins Explicit by Naming

It is highly recommended that mixins are named with a `...Mixin` suffix.

### 5. An ABC May Also Be a Mixin; The Reverse Is Not True

Because an ABC can implement concrete methods, it works as a mixin as well. An ABC also defines a type, which a mixin does not. An ABC can be the sole base class of any other class, while a mixin should never be subclasses alone, except by another, more specialised mixin.

One restriction applies to ABCs but not mixins: concrete methods implemented in ABCs should only collaberate with methods of the same ABC and its superclasses. 

This implies that concrete methods in an ABC are always for convenience. A user of the class should be able do everything the concrete methods do by calling other methods of the ABC.

### 6. Don't Subclass from More Than One Concrete Class

Concrete classes should have zero or at most one **concrete superclass**. All but one of the superclasses of a concrete class should be ABCs or mixins.

### 7. Provide Aggregate Classes to Users

If some combination of ABCs or mixins is particularly useful to client code, provide a class that brings them together in a sensible way. This is called an **aggregate class**.

### 8. "Favour Object Composition Over Class Inheritance."

**Note:** See the article [Inheritance and Composition: A Python OOP Guide @ RealPython.com](https://realpython.com/inheritance-composition-python/#whats-composition) for supplementary reading material related to this topic.

It is easy to overuse inheritance. However, favouring composition leads to more flexible designs. Composition implies a "has-a" relationship.

Instead of inheriting methods from multiple superclasses, instances of the subclass could hold references to these superclasses and invoke their methods.

Composition and delegation can replace the use of mixins to make behaviours available to different classes, but cannot replace the use of interface inheritance to define a hierarchy of types.